<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/netCDF4_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script um die netCDF4 Vorhersagewerte in einen Dataframe abzuspeichern

In [1]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive

# Mounten Sie Ihr Google Drive-Konto
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import der benötigten Module
import os
import netCDF4 as nc
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [4]:
# coordinates of the location
latitude = 50.7
longitude = 7.2
# time is given in hours from 1.1.1900 00:00
origin = datetime.datetime(1900, 1, 1, 0, 0, 0, 0)
# Monat der Vorhersage
dateiname = 'CSV_Cams'

In [5]:
drive_folder = '/content/drive/My Drive/Colab_Notebooks/netCDF4_Wetterdaten/'

In [6]:
def read_nc_files_in_google_drive(folder_path):
    nc_files = []  # Eine Liste, um NetCDF4-Datensätze zu speichern

    if os.path.exists(folder_path):
        # Durchsuchen Sie den Ordner nach Dateien
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # Überprüfen Sie, ob die Datei eine NetCDF4-Datei ist
            if file_name.endswith(".nc"):
                try:
                    # Öffnen Sie die NetCDF4-Datei und fügen Sie sie zur Liste hinzu
                    nc_file = nc.Dataset(file_path)
                    nc_files.append(nc_file)
                except Exception as e:
                    print(f"Fehler beim Öffnen von {file_name}: {str(e)}")

    return nc_files

In [7]:
netcdf_files = read_nc_files_in_google_drive(drive_folder)

In [10]:
latitudes = np.empty((1, 1))
longitudes = np.empty((1, 1))
time = np.empty((1, 1))
aod469 = np.empty((1, 1))
aod550 = np.empty((1, 1))
aod670 = np.empty((1, 1))
aod865 = np.empty((1, 1))
gtco3 = np.empty((1, 1))
uvbed = np.empty((1, 1))
uvbedcs = np.empty((1, 1))

In [11]:
# Zuordnen der Variabeln zu entsprechenden NP-Arrays
# fehlende Werte werden durch NaN Werte ersetzt, das mehrdimensionale Array wird zu einem eindimensonalen formatiert
for nc_file in netcdf_files:
    latitudes = np.append(latitudes, nc_file.variables['latitude'][0])
    longitudes = np.append(longitudes, nc_file.variables['longitude'][0])
    time = np.append(time, nc_file.variables['time'][:].filled(np.nan))
    aod469 = np.append(aod469, nc_file.variables['aod469'][:].filled(np.nan).reshape(-2))
    aod550 = np.append(aod550, nc_file.variables['aod550'][:].filled(np.nan).reshape(-2))
    aod670 = np.append(aod670, nc_file.variables['aod670'][:].filled(np.nan).reshape(-2))
    aod865 = np.append(aod865, nc_file.variables['aod865'][:].filled(np.nan).reshape(-2))
    gtco3 = np.append(gtco3, nc_file.variables['gtco3'][:].filled(np.nan).reshape(-2))
    uvbed = np.append(uvbed, nc_file.variables['uvbed'][:].filled(np.nan).reshape(-2))
    uvbedcs = np.append(uvbedcs, nc_file.variables['uvbedcs'][:].filled(np.nan).reshape(-2))

In [12]:
# richtige Datetime berechen
datetime = np.array([origin + datetime.timedelta(hours=int(i)) for i in time])

In [13]:
# Zuorden der NP-Arrays in einen Dataframe
df_cams = pd.DataFrame({'time': datetime, 'aod469': aod469, 'aod550': aod550,'aod670': aod670,'aod865': aod865,'aod865': gtco3,'uvbed': uvbed,'uvbedcs': uvbedcs,})

In [14]:
# Index des DF setzen
df_cams.set_index('time', inplace=True)

In [ ]:
df_cams

In [ ]:
# Resampling der Zeitreihe auf 2 Min. und interpolation der fehlenden Messwerte
df_cams_resampled = df_cams.resample('2T')
df_cams_interpolated = df_cams_resampled.interpolate(method='polynomial', order = 1)

In [ ]:
# Plotten eines Tages mit 2 Min. Auflösung und mit 1 Std. Auflösung
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].plot(df_cams_interpolated['aod865'][0:700])
axes[1].plot(df_cams['aod865'][0:25])

In [ ]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [ ]:
# Abspeichern des konvertierten DF als CSV
save_dataframe_to_drive(df_cams_interpolated, '/content/drive/My Drive/Colab_Notebooks/CSV_Vorhersage', dateiname)